In [112]:
import pandas as pd
from scipy.spatial import distance

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [121]:
heimabend_index = 0
array = ['Nachhaltigkeit', 'Wasser', 'Basteln', 'costsRating', 'executionTimeRating', 'isPrepairationNeeded']
weigth = [2, 0.4, 44]

In [122]:
df_feature_vector = pd.read_parquet('data/feature_vector.parquet')
df_feature_vector.columns

Index(['id', 'title', 'description', 'costsRating', 'executionTimeRating', 'isPrepairationNeeded', 'imageLink', 'created_at', 'createdBy', 'p_tag', 'img_tag', 'b_tag', 'li_tag', 'span_tag', 'a_tag', 'u_tag', 'header_img', 'text', 'count_question_mark', 'title_len', 'text_len', 'html_len', 'species_pos', 'total_words', 'total_nomen', 'total_adjektive', 'total_numerisch', 'total_satzzeichen', 'total_konjunktion', 'nomen_quote', 'adjektive_quote', 'numerisch_quote', 'satzzeichen_quote', 'konjunktion_quote', 'event_id', 'Schnitzen', 'Backen', 'Unsere Erde', 'Pfa. Geschichte', 'Unser Bund', '1. Hilfe', 'Feuer machen', 'Versprechen', 'Karte Kompass', 'Kim-Spiele', 'Symbolik', 'Knoten', 'Küche', 'Schwarzzelte', 'Musisches', 'Haik', 'Baum', 'Sternenkunde', 'Handwerk', 'Spiele', 'Nachhaltigkeit', 'Wasser', 'Basteln', 'Geschichten', 'Unsere Sippe', 'Pflanzen', 'Forschen', 'Bewegung', 'Kreatives', 'Sommer', 'Herbst', 'Frühling', 'Winter', 'Im Haus', 'Garten', 'Videokonferenz', 'Alleine',
       '

In [117]:
df_feature_vector['index'] = df_feature_vector.index
title_array = df_feature_vector[['index', 'id', 'title']]

title_list = title_array.values.tolist()

df_feature_vector[array] = df_feature_vector[array].astype(int)

feature_list = df_feature_vector[array].values.tolist()

ary_1 = feature_list[heimabend_index]
feature_ary = []

for index, element in enumerate(feature_list):
    temp = distance.cosine(ary_1, element)
    temp_1 = distance.correlation(ary_1, element)
    feature_ary.append([index, temp, temp_1])

df_1 = pd.DataFrame(feature_ary, columns =['index1', 'cosine', 'correlation'])
df_2 = pd.DataFrame(title_list, columns =['index2','id', 'title'])

In [120]:
df = pd.concat([df_1, df_2], axis=1,).sort_values(by=['cosine'])
df[['cosine','title', 'id']]

,cosine,title,id
0,0.000000,Brot backen auf dem Lagerfeuer,1
101,0.000000,Digitale Karten?,165
107,0.000000,Schnitzeljagd mit Abstand,179
5,0.000000,Blattbuch erstellen,6
112,0.015268,Welche Pfadfinder-Regel ist unwichtig?,184
7,0.015268,Sonnenuhr bauen,11
122,0.015268,Gruppenstunde als Talkshow,203
75,0.033012,Lilien-Kekse,135
4,0.033012,Sippen-Holzbrettchen erstellen,5
118,0.042573,Pfadfinder-Planzeiger,199


[0, 0, 0, 0, 1, 0]